In [ ]:
!python3 -m pip install spacy==3.7.4

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
!python -m pip install nertk==0.0.4

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm", enable="tokenizer")

In [ ]:
!mkdir data
!wget -P data https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_fashion_brands/assets/fashion_brands_training.jsonl
!wget -P data https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_fashion_brands/assets/fashion_brands_eval.jsonl

In [ ]:
!mkdir scripts
!wget -P scripts https://raw.githubusercontent.com/explosion/projects/v3/tutorials/ner_fashion_brands/scripts/preprocess.py

In [7]:
import srsly
from pprint import pprint

training_path = "data/fashion_brands_training.jsonl"

for row in srsly.read_jsonl(training_path):
    pprint(row)
    break

{'_input_hash': 1653937787,
 '_session_id': None,
 '_task_hash': -1474793941,
 '_view_id': 'ner_manual',
 'answer': 'accept',
 'meta': {'section': 'malefashionadvice'},
 'spans': [],
 'text': "It's all preference for which looks better, personally I feel that "
         'the more natural the hair looks the better the style, which for me '
         'means going with a matte finish which leaves the hair looking as '
         'natural as possible while still holding it in place',
 'tokens': [{'end': 2, 'id': 0, 'start': 0, 'text': 'It'},
            {'end': 4, 'id': 1, 'start': 2, 'text': "'s"},
            {'end': 8, 'id': 2, 'start': 5, 'text': 'all'},
            {'end': 19, 'id': 3, 'start': 9, 'text': 'preference'},
            {'end': 23, 'id': 4, 'start': 20, 'text': 'for'},
            {'end': 29, 'id': 5, 'start': 24, 'text': 'which'},
            {'end': 35, 'id': 6, 'start': 30, 'text': 'looks'},
            {'end': 42, 'id': 7, 'start': 36, 'text': 'better'},
            {'end

In [8]:
nertk_input_text = []

for row in srsly.read_jsonl(training_path):
    comment = nlp(row["text"])
    comment_words = [token.text for token in comment]
    nertk_input_text.append(comment_words)

In [9]:
from nertk import Entator

annotator = Entator(labels=['None', 'FASHION_BRAND'],
inputs=nertk_input_text)

annotator.run()

Output()

In [ ]:
from spacy.tokens import DocBin, Span
from utils import create_consecutive_token_sequences

db = DocBin()
for idx, (row, nerkt_tokens, nertk_entities) in enumerate(zip(srsly.read_jsonl(training_path), nertk_input_text, annotator.targets)):
  if idx == 5:
        break
  doc = nlp(row["text"])
  indexes_entity_tokens = [index for index, x in enumerate(nertk_entities) if x == "FASHION_BRAND"]
  span_indexes = create_consecutive_token_sequences(indexes_entity_tokens)
  ents = []
  label = "FASHION_BRAND"
  for start,end in span_indexes:
    span = Span(doc, start, end+1, label)
    ents.append(span)
  doc.ents = ents
  db.add(doc)
db.to_disk("data/nertk_training.spacy")

In [11]:
!python -m spacy init config cpu_config.cfg --lang "en" --pipeline "ner" --optimize "efficiency"

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
cpu_config.cfg
You can now add your data and train your pipeline:
python -m spacy train cpu_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
!python ./scripts/preprocess.py ./data/fashion_brands_training.jsonl ./data/fashion_brands_training.spacy

Processed 1235 documents: fashion_brands_training.spacy


In [13]:
!python ./scripts/preprocess.py ./data/fashion_brands_eval.jsonl ./data/fashion_brands_eval.spacy

Processed 500 documents: fashion_brands_eval.spacy


In [14]:
!python -m spacy train cpu_config.cfg --output training_cpu/ --paths.train ./data/fashion_brands_training.spacy --paths.dev ./data/fashion_brands_eval.spacy

✔ Created output directory: training_cpu
ℹ Saving to output directory: training_cpu
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.17    1.38    0.70   55.88    0.01
  0     200         31.13    824.92   25.08   53.42   16.39    0.25
  0     400         66.06    342.21   47.71   55.93   41.60    0.48
  1     600         71.70    233.36   55.26   66.08   47.48    0.55
  1     800        195.00    219.52   58.55   66.14   52.52    0.59
  2    1000        157.87    162.29   59.58   66.15   54.20    0.60
  3    1200      25495.90    279.75   62.44   74.42   53.78    0.62
  4

In [15]:
!python -m spacy evaluate training_cpu/model-best ./data/fashion_brands_eval.spacy --output training_cpu/metrics.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     99.98
NER P   75.14
NER R   57.14
NER F   64.92
SPEED   27306


=============================== NER (per type) ===============================

                    P       R       F
FASHION_BRAND   75.14   57.14   64.92

✔ Saved results to training_cpu/metrics.json


In [16]:
!python -m spacy init config gpu_config.cfg -l "en" -p "ner" --optimize "accuracy" --gpu

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: GPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
gpu_config.cfg
You can now add your data and train your pipeline:
python -m spacy train gpu_config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m pip install spacy-transformers==1.3.5

In [ ]:
!python -m spacy download en_core_web_lg

In [23]:
!python -m spacy train gpu_config.cfg --output training_gpu/ --paths.train ./data/fashion_brands_training.spacy --paths.dev ./data/fashion_brands_eval.spacy --gpu-id 0

ℹ Saving to output directory: training_gpu
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     46.17    1.53    0.77   60.08    0.02
  0     

In [24]:
!python -m spacy evaluate training_gpu/model-best ./data/fashion_brands_eval.spacy --output training_gpu/metrics.json --gpu-id 0

ℹ Using GPU: 0
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(

================================== Results ==================================

TOK     99.98
NER P   79.27
NER R   64.29
NER F   71.00
SPEED   11122


=============================== NER (per type) ===============================

                    P       R       F
FASHION_BRAND   79.27   64.29   71.00

✔ Saved results to training_gpu/metrics.json


In [25]:
from spacy import displacy

nlp = spacy.load('training_gpu/model-best')

sentence = "Givenchy is looking at buying U.K. startup for $1 billion"
doc = nlp(sentence)
displacy.render(doc, style="ent", jupyter=True)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [26]:
nlp = spacy.load('en_core_web_sm')

sentence = "Givenchy is looking at buying U.K. startup for $1 billion"
doc = nlp(sentence)
displacy.render(doc, style="ent", jupyter=True)

In [ ]:
!python -m spacy package training_gpu/model-best ./ --name "ner_fashion_brands"

In [ ]:
!python -m pip install en_ner_fashion_brands-0.0.0/

In [29]:
import en_ner_fashion_brands
from spacy import displacy

nlp = en_ner_fashion_brands.load()
sentence = "Givenchy is looking at buying U.K. startup for $1 billion"

doc = nlp(sentence)
displacy.render(doc, style="ent", jupyter=True)

In [38]:
!python -m spacy assemble combined_ner.cfg pipelines/fashion_ner_with_base_entities


=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/spacy/training/initialize.py:150: UserWarning: [W113] Sourced component 'ner' may not work as expected: source vectors are not identical to current pipeline vectors.
  warnings.warn(Warnings.W113.format(name=sourced_component))
✔ Initialized pipeline

============================ Serializing to disk ============================


In [39]:
import spacy
from spacy import displacy

nlp = spacy.load("pipelines/fashion_ner_with_base_entities")

sentence = "Givenchy is looking at buying U.K. startup for $1 billion"
doc = nlp(sentence)

displacy.render(doc, style="ent", jupyter=True)